<a href="https://colab.research.google.com/github/GaryPython/Cathay_LLM/blob/main/R4/R4_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# R4: 回家作業
- 目標:
    1. 請嘗試將下方語料庫中的文本透過量化技術轉為 embedding，並放入資料庫
    2. 檢索與 "怎麼提升模型效度" 最相關的10個文本以及相似度的分數(距離)
- 練習項目:
    1. chromadb 預設使用的大型語言模型為 `all-MiniLM-L6-v2`，由於該大型語言模型不支持中文，請嘗試將模型替換為 `infgrad/stella-base-zh-v3-1792d`，並對 embedding 進行量化
    2. 當語料庫的資料較多時，在將文本轉為向量或檢索時都需要花費許多時間，請嘗試使用 gpu 進行加速

In [1]:
!pip install langchain
!pip install -U langchain-community
!pip install chromadb
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [19]:
#from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from pprint import pprint # 導入 pprint 模組

## 語料庫
- 這是300則訓練機器學習模型時需要注意的事項，請將其整理為一個list，其中包含 300 個 element，每個 element 即為一個注意事項
- 注意，文本中可能存在重複的項目，請先對文本進行去除重複在接續後面的工作

In [3]:
corpus = '''1. **數據質量**：垃圾進，垃圾出。確保您的數據集是乾淨和準確的。
2. **數據平衡**：確保您的數據集在各類別之間是平衡的，以避免模型偏向某一類別。
3. **特徵選擇**：選擇與預測目標最相關的特徵，避免無關特徵導致模型過度擬合。
4. **數據預處理**：對數據進行標準化或歸一化，以提高模型的性能。
5. **模型選擇**：選擇最適合您數據和任務的模型，並不是所有模型都適用於所有問題。
6. **超參數調整**：使用交叉驗證來調整模型的超參數，以達到最佳性能。
7. **訓練/測試分割**：確保您的數據集被正確地分割成訓練集和測試集，以避免過度擬合。
8. **模型評估**：使用適當的評估指標來評估您的模型性能。
9. **迭代改進**：機器學習是一個迭代的過程，不斷優化您的模型以達到最佳性能。
10. **理解您的模型**：理解您的模型是如何做出預測的，這對於解釋模型的預測結果至關重要。
11. **避免過度擬合**：過度擬合是機器學習中的一個常見問題，可以通過正則化、早期停止等方法來避免。
12. **處理缺失值**：確保您的數據集中的缺失值被適當地處理。
13. **特徵工程**：創建新的特徵可以提高模型的性能。
14. **模型驗證**：使用新的、未見過的數據來驗證您的模型的性能。
15. **模型更新**：隨著新數據的收集，定期更新您的模型。
16. **數據安全和隱私**：在處理數據時，確保遵守所有相關的數據安全和隱私法規。
17. **模型偏見**：確保您的模型不包含任何不公平的偏見。
18. **模型解釋性**：選擇可以解釋的模型，以便理解模型的決策過程。
19. **模型部署**：確保您的模型可以在實際環境中運行。
20. **模型監控**：在模型部署後，持續監控其性能。
21. **數據收集**：確保您收集的數據可以反映您想要解決的問題。
22. **數據標註**：確保您的數據被正確地標註，以供模型學習。
23. **模型泛化**：確保您的模型可以泛化到新的、未見過的數據。
24. **模型複雜性**：避免使用過於複雜的模型，以避免過度擬合。
25. **模型穩定性**：確保您的模型在不同的數據集上都能保持穩定的性能。
26. **模型效率**：確保您的模型在合理的時間內運行。
27. **模型可擴展性**：確保您的模型可以處理大量的數據。
28. **模型多樣性**：嘗試使用不同的模型來解決同一個問題，以提高性能。
29. **模型魯棒性**：確保您的模型對輸入數據的小變化具有魯棒性。
30. **模型持久化**：確保您可以保存和加載訓練好的模型，以便在未來使用。
31. **數據探索**：在開始訓練模型之前，先進行數據探索，了解數據的分佈和特性。
32. **異常值處理**：確保您的數據集中的異常值被適當地處理。
33. **模型比較**：嘗試使用不同的模型並比較它們的性能。
34. **模型組合**：使用模型組合或集成學習可以提高預測性能。
35. **交叉驗證**：使用交叉驗證來評估模型的泛化能力。
36. **特徵編碼**：對類別特徵進行適當的編碼，如one-hot編碼。
37. **特徵縮放**：對數值特徵進行縮放，如最大最小縮放或標準化。
38. **數據增強**：使用數據增強來擴充您的數據集，特別是在圖像、聲音等領域。
39. **深度學習**：對於複雜的問題，可以考慮使用深度學習模型。
40. **轉移學習**：使用預訓練的模型並進行微調，可以節省訓練時間並提高性能。
41. **模型解釋工具**：使用模型解釋工具，如SHAP或LIME，來理解模型的決策過程。
42. **自動機器學習**：考慮使用自動機器學習工具，如AutoML，來自動化模型選擇和超參數調整的過程。
43. **模型保存和加載**：學會如何保存和加載訓練好的模型，以便在未來使用。
44. **模型版本控制**：使用模型版本控制工具，如MLflow，來追蹤模型的版本和性能。
45. **模型部署**：學習如何將模型部署到生產環境，如使用Docker容器。
46. **模型監控**：在模型部署後，定期監控模型的性能並進行必要的更新。
47. **模型測試**：撰寫單元測試和集成測試，以確保模型的正確性和穩定性。
48. **模型文檔**：撰寫清晰的模型文檔，包括模型的目的、使用的數據、特徵、模型結構、性能指標等。
49. **模型評估報告**：撰寫模型評估報告，包括模型的性能、錯誤分析、改進建議等。
50. **模型維護**：模型部署後，定期進行模型維護，包括更新數據、調整模型、改進性能等。
51. **模型審核**：進行模型審核，以確保模型的公平性、可解釋性、隱私性等。
52. **模型管道**：建立模型管道，包括數據預處理、特徵工程、模型訓練、模型評估等步驟。
53. **模型反饋迴路**：建立模型反饋迴路，收集模型預測的反饋，並用於改進模型。
54. **模型實驗**：進行模型實驗，比較不同模型、特徵、超參數的效果。
55. **模型優化**：進行模型優化，如特徵選擇、參數調整、模型選擇等。
56. **模型可視化**：進行模型可視化，如特徵重要性、模型結構、預測結果等。
57. **模型共享**：將模型共享給其他人或團隊使用，如使用模型服務API。
58. **模型再訓練**：當新數據可用時，進行模型的再訓練。
59. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
60. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。
61. **模型選擇**：選擇適合您的問題和數據的模型，例如，對於非線性問題，您可能需要使用決策樹或神經網路。
62. **特徵轉換**：考慮對特徵進行轉換，例如，對於偏態分佈的特徵，您可能需要進行對數轉換。
63. **數據清理**：確保您的數據是乾淨的，並且沒有錯誤或不一致的值。
64. **模型評估**：使用適當的評估指標來評估您的模型，例如，對於分類問題，您可能需要使用準確度、精確度、召回率或F1分數。
65. **模型解釋性**：選擇可以解釋的模型，這對於理解模型的工作原理和提供可解釋的預測是很重要的。
66. **模型泛化**：確保您的模型在新的、未見過的數據上也能表現良好，這是通過交叉驗證和測試集來評估的。
67. **模型優化**：使用網格搜索或隨機搜索來優化您的模型的超參數。
68. **模型更新**：隨著新數據的收集，定期更新您的模型。
69. **模型部署**：學習如何將您的模型部署到生產環境，以便在實際應用中使用。
70. **模型監控**：在模型部署後，持續監控其性能，並在需要時進行調整。
71. **模型驗證**：使用新的、未見過的數據來驗證您的模型的性能。
72. **模型保存和加載**：學會如何保存和加載訓練好的模型，以便在未來使用。
73. **模型文檔**：撰寫清晰的模型文檔，包括模型的目的、使用的數據、特徵、模型結構、性能指標等。
74. **模型評估報告**：撰寫模型評估報告，包括模型的性能、錯誤分析、改進建議等。
75. **模型維護**：模型部署後，定期進行模型維護，包括更新數據、調整模型、改進性能等。
76. **模型審核**：進行模型審核，以確保模型的公平性、可解釋性、隱私性等。
77. **模型管道**：建立模型管道，包括數據預處理、特徵工程、模型訓練、模型評估等步驟。
78. **模型反饋迴路**：建立模型反饋迴路，收集模型預測的反饋，並用於改進模型。
79. **模型實驗**：進行模型實驗，比較不同模型、特徵、超參數的效果。
80. **模型優化**：進行模型優化，如特徵選擇、參數調整、模型選擇等。
81. **模型可視化**：進行模型可視化，如特徵重要性、模型結構、預測結果等。
82. **模型共享**：將模型共享給其他人或團隊使用，如使用模型服務API。
83. **模型再訓練**：當新數據可用時，進行模型的再訓練。
84. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
85. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。
86. **模型選擇**：選擇適合您的問題和數據的模型，例如，對於非線性問題，您可能需要使用決策樹或神經網路。
87. **特徵轉換**：考慮對特徵進行轉換，例如，對於偏態分佈的特徵，您可能需要進行對數轉換。
88. **數據清理**：確保您的數據是乾淨的，並且沒有錯誤或不一致的值。
89. **模型評估**：使用適當的評估指標來評估您的模型，例如，對於分類問題，您可能需要使用準確度、精確度、召回率或F1分數。
90. **模型解釋性**：選擇可以解釋的模型，這對於理解模型的工作原理和提供可解釋的預測是很重要的。
91. **數據分析**：在開始訓練模型之前，先進行數據分析，了解數據的分佈和特性。
92. **異常值檢測**：確保您的數據集中的異常值被適當地處理。
93. **模型驗證**：使用交叉驗證來評估模型的泛化能力。
94. **模型比較**：嘗試使用不同的模型並比較它們的性能。
95. **模型組合**：使用模型組合或集成學習可以提高預測性能。
96. **特徵編碼**：對類別特徵進行適當的編碼，如one-hot編碼。
97. **特徵縮放**：對數值特徵進行縮放，如最大最小縮放或標準化。
98. **數據增強**：使用數據增強來擴充您的數據集，特別是在圖像、聲音等領域。
99. **深度學習**：對於複雜的問題，可以考慮使用深度學習模型。
100. **轉移學習**：使用預訓練的模型並進行微調，可以節省訓練時間並提高性能。
101. **模型解釋工具**：使用模型解釋工具，如SHAP或LIME，來理解模型的決策過程。
102. **自動機器學習**：考慮使用自動機器學習工具，如AutoML，來自動化模型選擇和超參數調整的過程。
103. **模型保存和加載**：學會如何保存和加載訓練好的模型，以便在未來使用。
104. **模型文檔**：撰寫清晰的模型文檔，包括模型的目的、使用的數據、特徵、模型結構、性能指標等。
105. **模型評估報告**：撰寫模型評估報告，包括模型的性能、錯誤分析、改進建議等。
106. **模型維護**：模型部署後，定期進行模型維護，包括更新數據、調整模型、改進性能等。
107. **模型審核**：進行模型審核，以確保模型的公平性、可解釋性、隱私性等。
108. **模型管道**：建立模型管道，包括數據預處理、特徵工程、模型訓練、模型評估等步驟。
109. **模型反饋迴路**：建立模型反饋迴路，收集模型預測的反饋，並用於改進模型。
110. **模型實驗**：進行模型實驗，比較不同模型、特徵、超參數的效果。
111. **模型優化**：進行模型優化，如特徵選擇、參數調整、模型選擇等。
112. **模型可視化**：進行模型可視化，如特徵重要性、模型結構、預測結果等。
113. **模型共享**：將模型共享給其他人或團隊使用，如使用模型服務API。
114. **模型再訓練**：當新數據可用時，進行模型的再訓練。
115. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
116. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。
117. **模型選擇**：選擇適合您的問題和數據的模型，例如，對於非線性問題，您可能需要使用決策樹或神經網路。
118. **特徵轉換**：考慮對特徵進行轉換，例如，對於偏態分佈的特徵，您可能需要進行對數轉換。
119. **數據清理**：確保您的數據是乾淨的，並且沒有錯誤或不一致的值。
120. **模型評估**：使用適當的評估指標來評估您的模型，例如，對於分類問題，您可能需要使用準確度、精確度、召回率或F1分數。
121. **數據探索**：在開始訓練模型之前，先進行數據探索，了解數據的分佈和特性。
122. **異常值處理**：確保您的數據集中的異常值被適當地處理。
123. **模型選擇**：選擇最適合您數據和任務的模型，並不是所有模型都適用於所有問題。
124. **超參數調整**：使用交叉驗證來調整模型的超參數，以達到最佳性能。
125. **訓練/測試分割**：確保您的數據集被正確地分割成訓練集和測試集，以避免過度擬合。
126. **模型評估**：使用適當的評估指標來評估您的模型性能。
127. **迭代改進**：機器學習是一個迭代的過程，不斷優化您的模型以達到最佳性能。
128. **理解您的模型**：理解您的模型是如何做出預測的，這對於解釋模型的預測結果至關重要。
129. **避免過度擬合**：過度擬合是機器學習中的一個常見問題，可以通過正則化、早期停止等方法來避免。
130. **處理缺失值**：確保您的數據集中的缺失值被適當地處理。
131. **特徵工程**：創建新的特徵可以提高模型的性能。
132. **模型驗證**：使用新的、未見過的數據來驗證您的模型的性能。
133. **模型更新**：隨著新數據的收集，定期更新您的模型。
134. **數據安全和隱私**：在處理數據時，確保遵守所有相關的數據安全和隱私法規。
135. **模型偏見**：確保您的模型不包含任何不公平的偏見。
136. **模型解釋性**：選擇可以解釋的模型，以便理解模型的決策過程。
137. **模型部署**：確保您的模型可以在實際環境中運行。
138. **模型監控**：在模型部署後，持續監控其性能。
139. **數據收集**：確保您收集的數據可以反映您想要解決的問題。
140. **數據標註**：確保您的數據被正確地標註，以供模型學習。
141. **模型泛化**：確保您的模型可以泛化到新的、未見過的數據。
142. **模型複雜性**：避免使用過於複雜的模型，以避免過度擬合。
143. **模型穩定性**：確保您的模型在不同的數據集上都能保持穩定的性能。
144. **模型效率**：確保您的模型在合理的時間內運行。
145. **模型可擴展性**：確保您的模型可以處理大量的數據。
146. **模型多樣性**：嘗試使用不同的模型來解決同一個問題，以提高性能。
147. **模型魯棒性**：確保您的模型對輸入數據的小變化具有魯棒性。
148. **模型持久化**：確保您可以保存和加載訓練好的模型，以便在未來使用。
149. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
150. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。
151. **數據探索**：在開始訓練模型之前，先進行數據探索，了解數據的分佈和特性。
152. **異常值處理**：確保您的數據集中的異常值被適當地處理。
153. **模型選擇**：選擇最適合您數據和任務的模型，並不是所有模型都適用於所有問題。
154. **超參數調整**：使用交叉驗證來調整模型的超參數，以達到最佳性能。
155. **訓練/測試分割**：確保您的數據集被正確地分割成訓練集和測試集，以避免過度擬合。
156. **模型評估**：使用適當的評估指標來評估您的模型性能。
157. **迭代改進**：機器學習是一個迭代的過程，不斷優化您的模型以達到最佳性能。
158. **理解您的模型**：理解您的模型是如何做出預測的，這對於解釋模型的預測結果至關重要。
159. **避免過度擬合**：過度擬合是機器學習中的一個常見問題，可以通過正則化、早期停止等方法來避免。
160. **處理缺失值**：確保您的數據集中的缺失值被適當地處理。
161. **特徵工程**：創建新的特徵可以提高模型的性能。
162. **模型驗證**：使用新的、未見過的數據來驗證您的模型的性能。
163. **模型更新**：隨著新數據的收集，定期更新您的模型。
164. **數據安全和隱私**：在處理數據時，確保遵守所有相關的數據安全和隱私法規。
165. **模型偏見**：確保您的模型不包含任何不公平的偏見。
166. **模型解釋性**：選擇可以解釋的模型，以便理解模型的決策過程。
167. **模型部署**：確保您的模型可以在實際環境中運行。
168. **模型監控**：在模型部署後，持續監控其性能。
169. **數據收集**：確保您收集的數據可以反映您想要解決的問題。
170. **數據標註**：確保您的數據被正確地標註，以供模型學習。
171. **模型泛化**：確保您的模型可以泛化到新的、未見過的數據。
172. **模型複雜性**：避免使用過於複雜的模型，以避免過度擬合。
173. **模型穩定性**：確保您的模型在不同的數據集上都能保持穩定的性能。
174. **模型效率**：確保您的模型在合理的時間內運行。
175. **模型可擴展性**：確保您的模型可以處理大量的數據。
176. **模型多樣性**：嘗試使用不同的模型來解決同一個問題，以提高性能。
177. **模型魯棒性**：確保您的模型對輸入數據的小變化具有魯棒性。
178. **模型持久化**：確保您可以保存和加載訓練好的模型，以便在未來使用。
179. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
180. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。
181. **數據探索**：在開始訓練模型之前，先進行數據探索，了解數據的分佈和特性。
182. **異常值處理**：確保您的數據集中的異常值被適當地處理。
183. **模型選擇**：選擇最適合您數據和任務的模型，並不是所有模型都適用於所有問題。
184. **超參數調整**：使用交叉驗證來調整模型的超參數，以達到最佳性能。
185. **訓練/測試分割**：確保您的數據集被正確地分割成訓練集和測試集，以避免過度擬合。
186. **模型評估**：使用適當的評估指標來評估您的模型性能。
187. **迭代改進**：機器學習是一個迭代的過程，不斷優化您的模型以達到最佳性能。
188. **理解您的模型**：理解您的模型是如何做出預測的，這對於解釋模型的預測結果至關重要。
189. **避免過度擬合**：過度擬合是機器學習中的一個常見問題，可以通過正則化、早期停止等方法來避免。
190. **處理缺失值**：確保您的數據集中的缺失值被適當地處理。
191. **特徵工程**：創建新的特徵可以提高模型的性能。
192. **模型驗證**：使用新的、未見過的數據來驗證您的模型的性能。
193. **模型更新**：隨著新數據的收集，定期更新您的模型。
194. **數據安全和隱私**：在處理數據時，確保遵守所有相關的數據安全和隱私法規。
195. **模型偏見**：確保您的模型不包含任何不公平的偏見。
196. **模型解釋性**：選擇可以解釋的模型，以便理解模型的決策過程。
197. **模型部署**：確保您的模型可以在實際環境中運行。
198. **模型監控**：在模型部署後，持續監控其性能。
199. **數據收集**：確保您收集的數據可以反映您想要解決的問題。
200. **數據標註**：確保您的數據被正確地標註，以供模型學習。
201. **模型泛化**：確保您的模型可以泛化到新的、未見過的數據。
202. **模型複雜性**：避免使用過於複雜的模型，以避免過度擬合。
203. **模型穩定性**：確保您的模型在不同的數據集上都能保持穩定的性能。
204. **模型效率**：確保您的模型在合理的時間內運行。
205. **模型可擴展性**：確保您的模型可以處理大量的數據。
206. **模型多樣性**：嘗試使用不同的模型來解決同一個問題，以提高性能。
207. **模型魯棒性**：確保您的模型對輸入數據的小變化具有魯棒性。
208. **模型持久化**：確保您可以保存和加載訓練好的模型，以便在未來使用。
209. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
210. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。
211. **數據探索**：在開始訓練模型之前，先進行數據探索，了解數據的分佈和特性。
212. **異常值處理**：確保您的數據集中的異常值被適當地處理。
213. **模型選擇**：選擇最適合您數據和任務的模型，並不是所有模型都適用於所有問題。
214. **超參數調整**：使用交叉驗證來調整模型的超參數，以達到最佳性能。
215. **訓練/測試分割**：確保您的數據集被正確地分割成訓練集和測試集，以避免過度擬合。
216. **模型評估**：使用適當的評估指標來評估您的模型性能。
217. **迭代改進**：機器學習是一個迭代的過程，不斷優化您的模型以達到最佳性能。
218. **理解您的模型**：理解您的模型是如何做出預測的，這對於解釋模型的預測結果至關重要。
219. **避免過度擬合**：過度擬合是機器學習中的一個常見問題，可以通過正則化、早期停止等方法來避免。
220. **處理缺失值**：確保您的數據集中的缺失值被適當地處理。
221. **特徵工程**：創建新的特徵可以提高模型的性能。
222. **模型驗證**：使用新的、未見過的數據來驗證您的模型的性能。
223. **模型更新**：隨著新數據的收集，定期更新您的模型。
224. **數據安全和隱私**：在處理數據時，確保遵守所有相關的數據安全和隱私法規。
225. **模型偏見**：確保您的模型不包含任何不公平的偏見。
226. **模型解釋性**：選擇可以解釋的模型，以便理解模型的決策過程。
227. **模型部署**：確保您的模型可以在實際環境中運行。
228. **模型監控**：在模型部署後，持續監控其性能。
229. **數據收集**：確保您收集的數據可以反映您想要解決的問題。
230. **數據標註**：確保您的數據被正確地標註，以供模型學習。
231. **模型泛化**：確保您的模型可以泛化到新的、未見過的數據。
232. **模型複雜性**：避免使用過於複雜的模型，以避免過度擬合。
233. **模型穩定性**：確保您的模型在不同的數據集上都能保持穩定的性能。
234. **模型效率**：確保您的模型在合理的時間內運行。
235. **模型可擴展性**：確保您的模型可以處理大量的數據。
236. **模型多樣性**：嘗試使用不同的模型來解決同一個問題，以提高性能。
237. **模型魯棒性**：確保您的模型對輸入數據的小變化具有魯棒性。
238. **模型持久化**：確保您可以保存和加載訓練好的模型，以便在未來使用。
239. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
240. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。
241. **數據探索**：在開始訓練模型之前，先進行數據探索，了解數據的分佈和特性。
242. **異常值處理**：確保您的數據集中的異常值被適當地處理。
243. **模型選擇**：選擇最適合您數據和任務的模型，並不是所有模型都適用於所有問題。
244. **超參數調整**：使用交叉驗證來調整模型的超參數，以達到最佳性能。
245. **訓練/測試分割**：確保您的數據集被正確地分割成訓練集和測試集，以避免過度擬合。
246. **模型評估**：使用適當的評估指標來評估您的模型性能。
247. **迭代改進**：機器學習是一個迭代的過程，不斷優化您的模型以達到最佳性能。
248. **理解您的模型**：理解您的模型是如何做出預測的，這對於解釋模型的預測結果至關重要。
249. **避免過度擬合**：過度擬合是機器學習中的一個常見問題，可以通過正則化、早期停止等方法來避免。
250. **處理缺失值**：確保您的數據集中的缺失值被適當地處理。
251. **特徵工程**：創建新的特徵可以提高模型的性能。
252. **模型驗證**：使用新的、未見過的數據來驗證您的模型的性能。
253. **模型更新**：隨著新數據的收集，定期更新您的模型。
254. **數據安全和隱私**：在處理數據時，確保遵守所有相關的數據安全和隱私法規。
255. **模型偏見**：確保您的模型不包含任何不公平的偏見。
256. **模型解釋性**：選擇可以解釋的模型，以便理解模型的決策過程。
257. **模型部署**：確保您的模型可以在實際環境中運行。
258. **模型監控**：在模型部署後，持續監控其性能。
259. **數據收集**：確保您收集的數據可以反映您想要解決的問題。
260. **數據標註**：確保您的數據被正確地標註，以供模型學習。
261. **模型泛化**：確保您的模型可以泛化到新的、未見過的數據。
262. **模型複雜性**：避免使用過於複雜的模型，以避免過度擬合。
263. **模型穩定性**：確保您的模型在不同的數據集上都能保持穩定的性能。
264. **模型效率**：確保您的模型在合理的時間內運行。
265. **模型可擴展性**：確保您的模型可以處理大量的數據。
266. **模型多樣性**：嘗試使用不同的模型來解決同一個問題，以提高性能。
267. **模型魯棒性**：確保您的模型對輸入數據的小變化具有魯棒性。
268. **模型持久化**：確保您可以保存和加載訓練好的模型，以便在未來使用。
269. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
270. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。
271. **數據探索**：在開始訓練模型之前，先進行數據探索，了解數據的分佈和特性。
272. **異常值處理**：確保您的數據集中的異常值被適當地處理。
273. **模型選擇**：選擇最適合您數據和任務的模型，並不是所有模型都適用於所有問題。
274. **超參數調整**：使用交叉驗證來調整模型的超參數，以達到最佳性能。
275. **訓練/測試分割**：確保您的數據集被正確地分割成訓練集和測試集，以避免過度擬合。
276. **模型評估**：使用適當的評估指標來評估您的模型性能。
277. **迭代改進**：機器學習是一個迭代的過程，不斷優化您的模型以達到最佳性能。
278. **理解您的模型**：理解您的模型是如何做出預測的，這對於解釋模型的預測結果至關重要。
279. **避免過度擬合**：過度擬合是機器學習中的一個常見問題，可以通過正則化、早期停止等方法來避免。
280. **處理缺失值**：確保您的數據集中的缺失值被適當地處理。
281. **特徵工程**：創建新的特徵可以提高模型的性能。
282. **模型驗證**：使用新的、未見過的數據來驗證您的模型的性能。
283. **模型更新**：隨著新數據的收集，定期更新您的模型。
284. **數據安全和隱私**：在處理數據時，確保遵守所有相關的數據安全和隱私法規。
285. **模型偏見**：確保您的模型不包含任何不公平的偏見。
286. **模型解釋性**：選擇可以解釋的模型，以便理解模型的決策過程。
287. **模型部署**：確保您的模型可以在實際環境中運行。
288. **模型監控**：在模型部署後，持續監控其性能。
289. **數據收集**：確保您收集的數據可以反映您想要解決的問題。
290. **數據標註**：確保您的數據被正確地標註，以供模型學習。
291. **模型泛化**：確保您的模型可以泛化到新的、未見過的數據。
292. **模型複雜性**：避免使用過於複雜的模型，以避免過度擬合。
293. **模型穩定性**：確保您的模型在不同的數據集上都能保持穩定的性能。
294. **模型效率**：確保您的模型在合理的時間內運行。
295. **模型可擴展性**：確保您的模型可以處理大量的數據。
296. **模型多樣性**：嘗試使用不同的模型來解決同一個問題，以提高性能。
297. **模型魯棒性**：確保您的模型對輸入數據的小變化具有魯棒性。
298. **模型持久化**：確保您可以保存和加載訓練好的模型，以便在未來使用。
299. **模型效能**：考慮模型的效能，如訓練時間、預測時間、模型大小等。
300. **模型安全性**：確保模型的安全性，防止模型被攻擊，如數據污染攻擊、模型竊取攻擊等。'''

In [15]:
corpus_lst = list(set(corpus.split('\n')[idx].split(' ')[-1].replace('*', '') for idx in range(300)))
print(f'corpus_lst shape: {len(corpus_lst)}')

corpus_lst shape: 72


In [16]:
corpus_lst

['模型部署：學習如何將模型部署到生產環境，如使用Docker容器。',
 '模型驗證：使用新的、未見過的數據來驗證您的模型的性能。',
 '數據平衡：確保您的數據集在各類別之間是平衡的，以避免模型偏向某一類別。',
 '模型評估報告：撰寫模型評估報告，包括模型的性能、錯誤分析、改進建議等。',
 '模型穩定性：確保您的模型在不同的數據集上都能保持穩定的性能。',
 '模型選擇：選擇適合您的問題和數據的模型，例如，對於非線性問題，您可能需要使用決策樹或神經網路。',
 '模型魯棒性：確保您的模型對輸入數據的小變化具有魯棒性。',
 '模型監控：在模型部署後，定期監控模型的性能並進行必要的更新。',
 '模型泛化：確保您的模型可以泛化到新的、未見過的數據。',
 '特徵編碼：對類別特徵進行適當的編碼，如one-hot編碼。',
 '模型優化：進行模型優化，如特徵選擇、參數調整、模型選擇等。',
 '模型測試：撰寫單元測試和集成測試，以確保模型的正確性和穩定性。',
 '模型監控：在模型部署後，持續監控其性能，並在需要時進行調整。',
 '異常值檢測：確保您的數據集中的異常值被適當地處理。',
 '異常值處理：確保您的數據集中的異常值被適當地處理。',
 '模型保存和加載：學會如何保存和加載訓練好的模型，以便在未來使用。',
 '超參數調整：使用交叉驗證來調整模型的超參數，以達到最佳性能。',
 '模型文檔：撰寫清晰的模型文檔，包括模型的目的、使用的數據、特徵、模型結構、性能指標等。',
 '模型組合：使用模型組合或集成學習可以提高預測性能。',
 '模型比較：嘗試使用不同的模型並比較它們的性能。',
 '模型效能：考慮模型的效能，如訓練時間、預測時間、模型大小等。',
 '模型解釋性：選擇可以解釋的模型，以便理解模型的決策過程。',
 '模型監控：在模型部署後，持續監控其性能。',
 '數據清理：確保您的數據是乾淨的，並且沒有錯誤或不一致的值。',
 '模型優化：使用網格搜索或隨機搜索來優化您的模型的超參數。',
 '模型可視化：進行模型可視化，如特徵重要性、模型結構、預測結果等。',
 '數據預處理：對數據進行標準化或歸一化，以提高模型的性能。',
 '模型驗證：使用交叉驗證來評估模型的泛化能力。',
 '特徵工程：創建新的特徵可以提高模型的性能。',
 '模型更新

## 將文本放入向量資料庫
- 替換為 `infgrad/stella-base-zh-v3-1792d` 的大型語言模型
- 將 embedding 做量化壓縮
- 使用 GPU 進行加速
- 放入向量資料庫

In [17]:
# 初始化 Embedding 模型

embedding_func = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={"precision":"binary"}) # 表示生成二進制形式的嵌入向量，節省存儲空間，但可能降低精度。

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/311 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/32.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

...

Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.51M [00:00<?, ?B/s]

In [20]:
# 加載文本數據
documents = [Document(page_content=text, metadata={'source': 'ml|_handbook'}) for text in corpus_lst]
pprint(documents)

[Document(metadata={'source': 'ml|_handbook'}, page_content='模型部署：學習如何將模型部署到生產環境，如使用Docker容器。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='模型驗證：使用新的、未見過的數據來驗證您的模型的性能。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='數據平衡：確保您的數據集在各類別之間是平衡的，以避免模型偏向某一類別。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='模型評估報告：撰寫模型評估報告，包括模型的性能、錯誤分析、改進建議等。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='模型穩定性：確保您的模型在不同的數據集上都能保持穩定的性能。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='模型選擇：選擇適合您的問題和數據的模型，例如，對於非線性問題，您可能需要使用決策樹或神經網路。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='模型魯棒性：確保您的模型對輸入數據的小變化具有魯棒性。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='模型監控：在模型部署後，定期監控模型的性能並進行必要的更新。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='模型泛化：確保您的模型可以泛化到新的、未見過的數據。'),
 Document(metadata={'source': 'ml|_handbook'}, page_content='特徵編碼：對類別特徵進行適當的編碼，如one-hot編碼。'),
 Document(metadata={'source': 'm

In [21]:
db = Chroma.from_documents(documents, embedding_func)

## 檢索資料
- 檢索與 "怎麼提升模型效度?" 最相關的10個文本以及相似度的分數(距離)

In [22]:
# query it
query = "怎麼提升模型效度?"
docs = db.similarity_search_with_score(query)
pprint(docs)

[(Document(metadata={'source': 'ml|_handbook'}, page_content='特徵工程：創建新的特徵可以提高模型的性能。'),
  1144288.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='轉移學習：使用預訓練的模型並進行微調，可以節省訓練時間並提高性能。'),
  1170663.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型優化：進行模型優化，如特徵選擇、參數調整、模型選擇等。'),
  1181568.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型驗證：使用新的、未見過的數據來驗證您的模型的性能。'),
  1283948.0)]


In [23]:
# query it
query = "怎麼提升模型效度?"
docs = db.similarity_search_with_score(query,k=10)
pprint(docs)

[(Document(metadata={'source': 'ml|_handbook'}, page_content='特徵工程：創建新的特徵可以提高模型的性能。'),
  1144288.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='轉移學習：使用預訓練的模型並進行微調，可以節省訓練時間並提高性能。'),
  1170663.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型優化：進行模型優化，如特徵選擇、參數調整、模型選擇等。'),
  1181568.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型驗證：使用新的、未見過的數據來驗證您的模型的性能。'),
  1283948.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型魯棒性：確保您的模型對輸入數據的小變化具有魯棒性。'),
  1306378.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型優化：使用網格搜索或隨機搜索來優化您的模型的超參數。'),
  1328076.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型效能：考慮模型的效能，如訓練時間、預測時間、模型大小等。'),
  1342288.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型組合：使用模型組合或集成學習可以提高預測性能。'),
  1356050.0),
 (Document(metadata={'source': 'ml|_handbook'}, page_content='模型評估報告：撰寫模型評估報告，包括模型的性能、錯誤分析、改進建議等。'),
  1364654.0),
 (Document(metadata={'sou

In [24]:

# 假设你已经有一个名为 `db` 的 Chroma 数据库对象
# 并且 `query` 是你的查询文本
query = "怎麼提升模型效度?"

# 使用相似性搜索查找相关文档及其相似度分数
results_with_scores = db.similarity_search_with_score(query,k=10)

# 处理和打印所有文档及其相似度
for i, (doc, score) in enumerate(results_with_scores):
    print(f"Document {i + 1}:")
    print(f"Score: {score}")
    print(f"Content: {doc.page_content}")
    print("-" * 50)


Document 1:
Score: 1144288.0
Content: 特徵工程：創建新的特徵可以提高模型的性能。
--------------------------------------------------
Document 2:
Score: 1170663.0
Content: 轉移學習：使用預訓練的模型並進行微調，可以節省訓練時間並提高性能。
--------------------------------------------------
Document 3:
Score: 1181568.0
Content: 模型優化：進行模型優化，如特徵選擇、參數調整、模型選擇等。
--------------------------------------------------
Document 4:
Score: 1283948.0
Content: 模型驗證：使用新的、未見過的數據來驗證您的模型的性能。
--------------------------------------------------
Document 5:
Score: 1306378.0
Content: 模型魯棒性：確保您的模型對輸入數據的小變化具有魯棒性。
--------------------------------------------------
Document 6:
Score: 1328076.0
Content: 模型優化：使用網格搜索或隨機搜索來優化您的模型的超參數。
--------------------------------------------------
Document 7:
Score: 1342288.0
Content: 模型效能：考慮模型的效能，如訓練時間、預測時間、模型大小等。
--------------------------------------------------
Document 8:
Score: 1356050.0
Content: 模型組合：使用模型組合或集成學習可以提高預測性能。
--------------------------------------------------
Document 9:
Score: 1364654.0
Content: 模型評估報告：撰寫模型評估報告，包括模型的性

In [10]:
for doc, score in docs:
    print(f"文檔: {doc}\n相似度得分: {score}\n")

文檔: page_content='特徵工程：創建新的特徵可以提高模型的性能。' metadata={'source': 'ml|_handbook'}
相似度得分: 1144288.0

文檔: page_content='特徵工程：創建新的特徵可以提高模型的性能。' metadata={'source': 'ml|_handbook'}
相似度得分: 1144288.0

文檔: page_content='特徵工程：創建新的特徵可以提高模型的性能。' metadata={'source': 'ml|_handbook'}
相似度得分: 1144288.0

文檔: page_content='特徵工程：創建新的特徵可以提高模型的性能。' metadata={'source': 'ml|_handbook'}
相似度得分: 1144288.0



In [ ]:
docs

[(Document(page_content='特徵工程：創建新的特徵可以提高模型的性能。', metadata={'source': 'ml|_handbook'}),
  1144288.0),
 (Document(page_content='轉移學習：使用預訓練的模型並進行微調，可以節省訓練時間並提高性能。', metadata={'source': 'ml|_handbook'}),
  1170663.0),
 (Document(page_content='模型優化：進行模型優化，如特徵選擇、參數調整、模型選擇等。', metadata={'source': 'ml|_handbook'}),
  1181568.0),
 (Document(page_content='模型驗證：使用新的、未見過的數據來驗證您的模型的性能。', metadata={'source': 'ml|_handbook'}),
  1283948.0),
 (Document(page_content='模型魯棒性：確保您的模型對輸入數據的小變化具有魯棒性。', metadata={'source': 'ml|_handbook'}),
  1306378.0),
 (Document(page_content='模型優化：使用網格搜索或隨機搜索來優化您的模型的超參數。', metadata={'source': 'ml|_handbook'}),
  1328076.0),
 (Document(page_content='模型監控：在模型部署後，定期監控模型的性能並進行必要的更新。', metadata={'source': 'ml|_handbook'}),
  1329594.0),
 (Document(page_content='模型比較：嘗試使用不同的模型並比較它們的性能。', metadata={'source': 'ml|_handbook'}),
  1335658.0),
 (Document(page_content='模型效能：考慮模型的效能，如訓練時間、預測時間、模型大小等。', metadata={'source': 'ml|_handbook'}),
  1342288.0),
 (Document(page_content='模型組合：使用